# Softmax

Avec cet exercice, vous serez appelé à:

- implanter un réseau de neurones de classification linéaire à 10 classes avec sortie **Softmax**
- implanter une fonction de perte (**loss function**) vectorisée de type **entropie croisée**
- implanter des **gradients analytiques rétropropagés** vectorisés
- utilisation de **gradients numériques** pour vérifier vos gradients analytiques
- utiliser un ensemble de validation pour ajuster le **taux d'apprentissage** et le paramètre de **régularisation**
- implanter une descente de gradient stochastique (**SGD**)
- **visualiser** les poids appris


In [1]:
import random
import numpy as np
from ift725.data_utils import load_CIFAR10
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'ift725/datasets/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
  
    # subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]

    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))

    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image

    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])

    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

## Classifieur Softmax

Le code pour cette section réside dans **ift725/classifiers/softmax.py**. 


In [ ]:
##############################################################################
# TODO: Implémenter le calcul de perte de façon naive avec des boucles dans  #
#  la fonction softmax_naive_loss_function située dans le fichier            #
#  ift725.classifiers.softmax.                                               #
##############################################################################

from ift725.classifiers.softmax import softmax_naive_loss_function
import time

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_naive_loss_function(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

## Question 1:
Pourquoi s'attend-on que la loss soit approximativement -np.log(0.1))?

**Votre réponse:** Lors d'une initialisation aléatoire des paramettres, la sortie du système est similaire à du bruit. 
Ainsi avec 10 classes, la softmax donne des probabilités équivalentes pour toutes les classes et l'entropie croisée nous donne 
-np.log(1/nb de classe) soit -np.log(0.1).

In [ ]:
##############################################################################
# TODO: Compléter le calcul du gradient de façon naive avec des boucles dans #
#  la fonction softmax_naive_loss_function située dans le fichier            #
#  ift725.classifiers.softmax.                                               #
##############################################################################
loss, grad = softmax_naive_loss_function(W, X_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from ift725.gradient_check import check_gradient_sparse
f = lambda w: softmax_naive_loss_function(w, X_dev, y_dev, 0.0)[0]
grad_numerical = check_gradient_sparse(f, W, grad, 10)

# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_naive_loss_function(W, X_dev, y_dev, 1e2)
f = lambda w: softmax_naive_loss_function(w, X_dev, y_dev, 1e2)[0]
grad_numerical = check_gradient_sparse(f, W, grad, 10)

### Question 2:  
Les 2 prochaines étapes consistent à implanter le calcul de la loss et du gradient de la loss de façon **linéarisée**.  Avant de commencer à coder, veuillez donner la formule de la loss et du gradient pour une mini-batch de 500 données contenue dans le tableau 

$$X \in R^{500\times 3073}$$

et une matrice de poids $$W \in R^{3073\times 10}$$ 

où 3073 est la dimensionnalité des données et 10 est le nombre de classes.

**Votre Réponse:** 

$$Loss = -\mathrm{tr}\left(T^T.\ln(X.W)\right)$$ avec la matrice des cibles $$ T \in \mathbb{R}^{500\times 10} $$ et la multiplication termes à termes "$*$".

$$dW = (Y-T).X^T$$ avec la prédiction $$Y $$

**NOTE IMPORTANT** : la réponse à cette question ne contient aucune boucle, seulement des multiplications matricielles et ventorielles

In [9]:
##############################################################################
# TODO: Implémenter le calcul de perte et du gradient de façon vectorielle   #
#  dans la fonction softmax_vectorized_loss_function située dans le fichier  #
#  ift725.classifiers.softmax.                                               #
# Les deux versions devraient calculer les mêmes résultats, mais la version  #
# vectorielle devrait être beaucoup plus rapide.                             #
##############################################################################
tic = time.time()
loss_naive, grad_naive = softmax_naive_loss_function(W, X_dev, y_dev, 0.00001)
toc = time.time()
print(X_dev.shape)
print('naive loss: %e computed in %fs' % (loss_naive, toc - tic))

from ift725.classifiers.softmax import softmax_vectorized_loss_function
tic = time.time()
loss_vectorized, grad_vectorized = softmax_vectorized_loss_function(W, X_dev, y_dev, 0.00001)
toc = time.time()
print('vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# As we did for the SVM, we use the Frobenius norm to compare the two versions
# of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Loss difference: %f' % np.abs(loss_naive - loss_vectorized))
print('Gradient difference: %f' % grad_difference)

(500, 3073)
naive loss: 2.367888e+00 computed in 43.775565s
(500, 3073)
(3073, 10)
(500,)
c (10, 10)[[ -1.66465219  -3.90302539   5.60731614   6.5436795  -14.89376432
   -8.01611016  -3.67358412   1.60233524  -2.62666778   1.13122374]
 [  2.21724062   2.29691801   1.90048345   0.27614564  -2.07876157
   -2.55831921  -2.49600449   0.97516674  -2.0258803   -0.47376094]
 [  0.29456512  -4.53327987  -2.26042607   6.94730999  -3.71856061
    2.44206499   0.76676638   1.48539359   1.93491696   2.30876189]
 [ -1.16072049   2.10451185  -8.60321314  -4.50712952  13.09080357
    7.61662664   5.80558883  -1.24578973   5.59815324   4.44573509]
 [ -7.29854571  -2.05902593  -5.91827443  -3.08729037   9.20928776
    2.91228466   3.91173506  -0.32473863   3.69378668   7.26176991]
 [ -1.36664455   4.93129879  -8.44978591  -6.11443283  10.40346275
    5.79139299   3.19736072  -5.09642828   0.68812073   3.27208013]
 [ -1.57421804   3.46000205  -3.78368364  -6.01294112  17.91420324
    8.39789256   3.0986

In [7]:
from ift725.classifiers import Softmax
import itertools as it
results = {}
best_val = -1
best_softmax = None
learning_rates = np.linspace(1e-7, 5e-7, 5)
regularization_strengths = np.linspace(1e4, 1e5, 3)
best_loss_history = None

##############################################################################
# TODO: Utilisez l'ensemble de validation pour régler les hyper-paramètres   #
#  (force de régularisation et vitesse d'apprentissage). Vous devez          #
#  expérimenter différentes plages de valeurs pour les taux d'apprentissage  #
#  et les forces de régularisation; si vous êtes prudent, vous devriez       #
#  pouvoir obtenir une précision de classification supérieur à 0.36 sur      #
#  l'ensemble de validation.                                                 #
##############################################################################
for lr, reg in it.product(learning_rates, regularization_strengths):
    classifier = Softmax()
    loss_history = classifier.train(X_train, y_train, learning_rate=lr, reg=reg, num_iters=1000)
    y_train_pred = classifier.predict(X_train)
    y_val_pred = classifier.predict(X_val)
    print("y_train_pred " + str(y_train_pred.shape) + str(y_train_pred))
    print("y_train " + str(y_train.shape) + str(y_train))
    acc_train = np.mean(y_train == y_train_pred)
    acc_val = np.mean(y_val == y_val_pred)
    results[(lr, reg)] = (acc_train, acc_val)
    
    if acc_val > best_val:
        print('best acc %f : lr=%f reg=%f' %(acc_val, lr, reg))
        best_val = acc_val
        best_softmax = classifier
        best_loss_history = loss_history
################################################################################
#                             FIN DE VOTRE CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

plt.plot(best_loss_history)
plt.xlabel('Iteration number')
plt.ylabel('Loss value')
plt.show()

IndexError: index 200 is out of bounds for axis 0 with size 200

In [ ]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy) )

In [ ]:
# Visualize the learned weights for each class
w = best_softmax.W[:-1,:] # strip out the bias
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)

    # Rescale the weights to be between 0 and 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])